In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm import tqdm
import plotly.graph_objects as go
from plotting import CandlePlot
from data import Data
pd.set_option("display.max_columns", None)

In [3]:
MOVE_COUNT = 10
PIPS = 20

In [4]:
data_path = '../data/'
# out_path = 'D:/Trading/ml4t-data/'
instruments = "../data/instruments.json"
out_file = f'D:/Trading/ml4t-data/hist.{MOVE_COUNT}moves.{PIPS}pips..csv'
d = Data(source="../data/EUR_USD_M5.pkl", ticker='EUR_USD', instruments=instruments)

In [5]:
our_cols = ['time', 'mid_c']

add_cols = dict(
            pip_returns=float,
            move_1_begin=int,
            move_1_pips=float,
            move_1_direction=object,
            move_2_begin=int,
            move_2_pips=float,
            move_2_direction=object,
            move_3_begin=int,
            move_3_pips=float,
            move_3_direction=object,
            move_4_begin=int,
            move_4_pips=float,
            move_4_direction=object,
            move_5_begin=int,
            move_5_pips=float,
            move_5_direction=object,
            move_6_begin=int,
            move_6_pips=float,
            move_6_direction=object,
            move_7_begin=int,
            move_7_pips=float,
            move_7_direction=object,
            move_8_begin=int,
            move_8_pips=float,
            move_8_direction=object,
            move_9_begin=int,
            move_9_pips=float,
            move_9_direction=object,
            move_10_begin=int,
            move_10_pips=float,
            move_10_direction=object,
            move_11_begin=int,
            move_11_pips=float,
            move_11_direction=object,
            move_12_begin=int,
            move_12_pips=float,
            move_12_direction=object,
            move_13_begin=int,
            move_13_pips=float,
            move_13_direction=object,
            move_14_begin=int,
            move_14_pips=float,
            move_14_direction=object,
            move_15_begin=int,
            move_15_pips=float,
            move_15_direction=object,
            move_16_begin=int,
            move_16_pips=float,
            move_16_direction=object,
            move_17_begin=int,
            move_17_pips=float,
            move_17_direction=object,
            move_18_begin=int,
            move_18_pips=float,
            move_18_direction=object,
            move_19_begin=int,
            move_19_pips=float,
            move_19_direction=object,
            move_20_begin=int,
            move_20_pips=float,
            move_20_direction=object,
            moves_trend=object
        )

candles = d.datalen
d.prepare_fast_data(name='ana2', start=0, end=candles, cols=our_cols, add_cols=add_cols)

In [6]:
def calc_pip_returns(price: np.ndarray) -> np.ndarray:
    next_price = np.append(price[1:], np.nan)
    return (next_price - price) * pow(10, 4)

def calc_pip_returns(price: pd.Series) -> pd.Series:
    prev_price = price.shift(1)
    return (price - prev_price) * pow(10, 4)
# d.update_fdata('pip_returns', value=calc_pip_returns(d.fdata('mid_c')))
d.df['ana2']['pip_returns'] = calc_pip_returns(d.df['ana2']['mid_c'])

In [7]:
d.df['ana2'].head()

,time,mid_c,pip_returns,move_1_begin,move_1_pips,move_1_direction,move_2_begin,move_2_pips,move_2_direction,move_3_begin,move_3_pips,move_3_direction,move_4_begin,move_4_pips,move_4_direction,move_5_begin,move_5_pips,move_5_direction,move_6_begin,move_6_pips,move_6_direction,move_7_begin,move_7_pips,move_7_direction,move_8_begin,move_8_pips,move_8_direction,move_9_begin,move_9_pips,move_9_direction,move_10_begin,move_10_pips,move_10_direction,move_11_begin,move_11_pips,move_11_direction,move_12_begin,move_12_pips,move_12_direction,move_13_begin,move_13_pips,move_13_direction,move_14_begin,move_14_pips,move_14_direction,move_15_begin,move_15_pips,move_15_direction,move_16_begin,move_16_pips,move_16_direction,move_17_begin,move_17_pips,move_17_direction,move_18_begin,move_18_pips,move_18_direction,move_19_begin,move_19_pips,move_19_direction,move_20_begin,move_20_pips,move_20_direction,moves_trend
0,2016-01-07 00:00:00,1.07786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-07 00:05:00,1.07810,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-07 00:10:00,1.07828,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07 00:15:00,1.07798,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-07 00:20:00,1.07790,-0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
d.df['ana2']['pip_returns'].describe(percentiles=[0.0001, 0.001, 0.005, .01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99, .995, .999, .9999])

count     594302.000000
mean           0.000435
std            3.273659
min         -119.400000
0.01%        -39.213980
0.1%         -18.500000
0.5%         -11.300000
1%            -9.000000
5%            -4.700000
10%           -3.200000
20%           -1.800000
30%           -1.000000
40%           -0.400000
50%            0.000000
60%            0.400000
70%            1.000000
80%            1.800000
90%            3.200000
95%            4.600000
99%            8.800000
99.5%         11.200000
99.9%         19.600000
99.99%        42.055920
max          166.200000
Name: pip_returns, dtype: float64

In [9]:
1.662000e+02 - 166

0.19999999999998863

In [10]:
d.df['ana2']['pip_returns'].max()

166.19999999999857

In [11]:
# d.update_fdata('pip_returns', index=1, value=0)

In [12]:
d.df['ana2'].head()

,time,mid_c,pip_returns,move_1_begin,move_1_pips,move_1_direction,move_2_begin,move_2_pips,move_2_direction,move_3_begin,move_3_pips,move_3_direction,move_4_begin,move_4_pips,move_4_direction,move_5_begin,move_5_pips,move_5_direction,move_6_begin,move_6_pips,move_6_direction,move_7_begin,move_7_pips,move_7_direction,move_8_begin,move_8_pips,move_8_direction,move_9_begin,move_9_pips,move_9_direction,move_10_begin,move_10_pips,move_10_direction,move_11_begin,move_11_pips,move_11_direction,move_12_begin,move_12_pips,move_12_direction,move_13_begin,move_13_pips,move_13_direction,move_14_begin,move_14_pips,move_14_direction,move_15_begin,move_15_pips,move_15_direction,move_16_begin,move_16_pips,move_16_direction,move_17_begin,move_17_pips,move_17_direction,move_18_begin,move_18_pips,move_18_direction,move_19_begin,move_19_pips,move_19_direction,move_20_begin,move_20_pips,move_20_direction,moves_trend
0,2016-01-07 00:00:00,1.07786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-07 00:05:00,1.07810,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-07 00:10:00,1.07828,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07 00:15:00,1.07798,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-07 00:20:00,1.07790,-0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# d.fastdf

In [14]:
# d.update_fdata('pip_returns', index=2, value=10)

In [15]:
# d.fdata('pip_returns', 2)

In [16]:
d.df['ana2']

,time,mid_c,pip_returns,move_1_begin,move_1_pips,move_1_direction,move_2_begin,move_2_pips,move_2_direction,move_3_begin,move_3_pips,move_3_direction,move_4_begin,move_4_pips,move_4_direction,move_5_begin,move_5_pips,move_5_direction,move_6_begin,move_6_pips,move_6_direction,move_7_begin,move_7_pips,move_7_direction,move_8_begin,move_8_pips,move_8_direction,move_9_begin,move_9_pips,move_9_direction,move_10_begin,move_10_pips,move_10_direction,move_11_begin,move_11_pips,move_11_direction,move_12_begin,move_12_pips,move_12_direction,move_13_begin,move_13_pips,move_13_direction,move_14_begin,move_14_pips,move_14_direction,move_15_begin,move_15_pips,move_15_direction,move_16_begin,move_16_pips,move_16_direction,move_17_begin,move_17_pips,move_17_direction,move_18_begin,move_18_pips,move_18_direction,move_19_begin,move_19_pips,move_19_direction,move_20_begin,move_20_pips,move_20_direction,moves_trend
0,2016-01-07 00:00:00,1.07786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-07 00:05:00,1.07810,2.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-07 00:10:00,1.07828,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-07 00:15:00,1.07798,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-07 00:20:00,1.07790,-0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594298,2023-12-29 21:35:00,1.10379,1.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594299,2023-12-29 21:40:00,1.10379,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594300,2023-12-29 21:45:00,1.10385,0.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
594301,2023-12-29 21:50:00,1.10392,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for i in tqdm(range(candles), desc=" Processing... "):
    pivot = i
    for j in range(MOVE_COUNT):
        # print(i, j, pivot)
        if pivot + 1 > candles:
            break
        close_price, forward_close_prices = d.fdata('mid_c', index=pivot), d.fdata('mid_c', index=pivot+1, rows=5000)
        pip_returns = np.round((forward_close_prices - close_price) * pow(10, 4), 2)
        d.update_fdata(f'move_{j+1}_begin', i, pivot+1)
        # target = pip_returns > PIPS or pip_returns < -PIPS
        targetup = pip_returns >= PIPS
        targetdown = pip_returns <= -PIPS
        try:
            pivotup = np.where(targetup)[0][0]
        except:
            pivotup = -1
        try:
            pivotdown = np.where(targetdown)[0][0]
        except:
            pivotdown = -1
        if pivotup == -1 and pivotdown == -1:
            break
        elif pivotup >= 0 and pivotdown >= 0:
            returns_pivot = min(pivotup, pivotdown)
        elif pivotup == -1:
            returns_pivot = pivotdown
        elif pivotdown == -1:
            returns_pivot = pivotup

        # print(i, pivot, pivotup, pivotdown, pip_returns[pivot])
        # try:
        d.update_fdata(f'move_{j+1}_pips', i, pip_returns[returns_pivot])
        d.update_fdata(f'move_{j+1}_direction', i, '1' if pip_returns[returns_pivot] > 0 else '0')
        # except:
        #     passp
        pivot = pivot + returns_pivot + 1 # Update pivot to close timestamp: Close means when PIPS target met
            

 Processing... :  13%|█▎        | 80150/594303 [02:56<18:08, 472.54it/s]

In [ ]:
d.df['ana2']['moves_trend'].fillna("'", inplace=True)
for i in range(MOVE_COUNT):
    d.df['ana2']['moves_trend'] = d.df['ana2']['moves_trend'] + d.df['ana2'][f'move_{i+1}_direction'].astype(str)

In [ ]:
d.df['ana2']['moves_trend'] = d.df['ana2']['moves_trend'].str.replace('nan', '')

In [ ]:
d.df['ana2'].head()

In [ ]:
d.df['ana2'].info()

In [ ]:
d.df['ana2'].to_csv(out_file)

In [ ]:
# cp = CandlePlot(d.df['ana2'][['time','mid_c']][41440:415258], candles=False)
# cp.show_plot()
cp = CandlePlot(d.df['ana2'].iloc[41440:415258], candles=False)
# cp.create_candle_fig()
cp.show_plot(line_traces=['mid_c'])